In [39]:
import csv
from bs4 import BeautifulSoup
from msedge.selenium_tools import Edge, EdgeOptions
import pandas as pd
import csv
import json
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as uReq
import pandas as pd
import time

In [40]:
test = []
names = []
flipPrices = []
prodNames = []
info,price = [],[]
url = "https://flipkart.com/search?q="
# query to search for.
q = input("Enter product name (Searching on flipkart)")
q = q.replace(" ","+")
#     q = input("Enter a query: ")
file_name = q.replace(" ", "_")
# response recieved in bytes
resp = requests.get(url+q)
# parsing response content using BeautifulSoup class, so that we can perform operations on it.
parsed_html = bs(resp.content, 'html.parser')
# data cleaning
raw_data = parsed_html.find("script", attrs={"id":"is_script"})
data = raw_data.contents[0].replace("window.__INITIAL_STATE__ = ","").replace(";","")
json_data = json.loads(data)
req_data = json_data["pageDataV4"]["page"]["data"]["10003"]   #[10]["widget"]["data"]["products"][3]["productInfo"]
#req_json_data = json_data["seoMeta"]["answerBox"]["data"]["renderableComponents"][0]["value"]["data"]

In [41]:
data_list = []
# print(len(req_data))
try:
    for i in range(1, len(req_data)):
        d = {}
        jd = req_data[i]["widget"]["data"]["products"]
        # print(len(jd))
        # print("i: ", i, end="\n")
        for j in range(len(jd)):
            jd2 = jd[j]["productInfo"]["value"]

            d["title"] = jd2["titles"]["title"]
            d["keySpecs"] = jd2["keySpecs"]
            d["rating"] = jd2["rating"]["average"]
            d["ratingCount"] = jd2["rating"]["count"]
            d["price"] = jd2["pricing"]["finalPrice"]["value"]
            #d["warranty"] = jd2["warrantySummary"]
            d["url"] = jd2["smartUrl"]

            # You can uncomment below lines if you want to print json output on terminal

            # print("Title: ",jd2["titles"]["title"],end="\n")
            # print("key specs: ", jd2["keySpecs"], end="\n")
            # print("Rating: ", jd2["rating"]["average"], end="\n")
            # print("Total ratings: ", jd2["rating"]["count"], end="\n")
            # print("Price: ", jd2["pricing"]["finalPrice"]["value"],end="\n")
            # print("warranty: ", jd2["warrantySummary"], end="\n")
            # print("Smart url: ", jd2["smartUrl"], end="\n")
        data_list.append(d)
        
except:
    pass 
# dumping data to result.json file
    # print(list(data_list))
with open("flipkart"+'.json', 'w') as fp:
    json.dump(data_list, fp)

# Now let us write our data to csv file
data_file = open("flipkart"+'.csv', 'w') 

# create the csv writer object 
csv_writer = csv.writer(data_file) 

# Counter variable used for writing  
# headers to the CSV file 
count = 0




In [42]:
for data in data_list:
    if count == 0: 

        # Writing headers of CSV file 
        header = data.keys() 
        csv_writer.writerow(header) 
        count += 1
        
    # Writing data of CSV file 

    csv_writer.writerow(data.values()) 


with open('flipkart.csv') as csv_file:
    reader = csv.reader(csv_file, delimiter=',')
    rows = list(reader)
i,j = 0,2
while i in range(len(rows)):
    try:
        name = rows[j][0]
        #name = " ".join(name.split(' ')[0:2])
        #print(name)
        #print("name = ",name)
        names.append(name)
        i += 1
        j += 2
    except:
            break


In [43]:
print("Best results",len(names))
#print(names,len(names))
if len(names) <= 10:
    flipkart_url = "https://www.flipkart.com/search?q=" + q
    print(flipkart_url)
    uClient = uReq(flipkart_url)
    flipkartPage = uClient.read()
    uClient.close()
    flipkart_html = bs(flipkartPage, "html.parser")
    bigboxes = flipkart_html.find_all("a", {"class": "s1Q9rs"})
    soup = BeautifulSoup(flipkartPage, 'html.parser')
    info = soup.select("[class~=s1Q9rs]")
    flipPrices = soup.select("[class =_30jeq3]")
    prodNames = [i.get('title') for i in info]
    names = prodNames
    df = pd.DataFrame(list(zip(prodNames, flipPrices)), 
                columns =['product_name', 'Flipkart_price']) 
    df.to_csv('test.csv')
    print(df)

else:
    with open('flipkart.csv') as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        rows = list(reader)

Best results 0
https://www.flipkart.com/search?q=clocks
                                         product_name Flipkart_price
0      R&P ENTERPRISE Analog 70 cm X 60 cm Wall Clock         [₹399]
1   ANNORA INTERNATIONAL Analog 25.5 cm X 25.5 cm ...         [₹335]
2               Kadio Analog 20 cm X 20 cm Wall Clock         [₹179]
3               Kadio Analog 20 cm X 20 cm Wall Clock         [₹179]
4              AJANTA Analog 32 cm X 32 cm Wall Clock         [₹474]
5          AJANTA Analog 26.5 cm X 26.5 cm Wall Clock         [₹268]
6          AJANTA Analog 20.5 cm X 20.5 cm Wall Clock         [₹279]
7   ANNORA INTERNATIONAL Analog 25 cm X 25 cm Wall...         [₹335]
8               Kadio Analog 33 cm X 24 cm Wall Clock         [₹249]
9           CAPIO ART Analog 26 cm X 26 cm Wall Clock         [₹292]
10  ANNORA INTERNATIONAL Analog 25.5 cm X 25.5 cm ...         [₹335]
11  pawan sut art and craft Analog 25 cm X 25 cm W...         [₹329]
12          Kadio Analog 24.5 cm X 24.5 cm Wall

In [44]:
    i,j = 0,2
    while i in range(len(rows)):
        try:
            price = rows[j][4]
            #price = price[i].text
            #print("price = ",price)
            flipPrices.append(price)
            i += 1
            j += 2
        except:
            break
    df = pd.DataFrame(list(zip(names, flipPrices)), 
            columns =['Product_name', 'Flipkart_price'])

    print(df)
    df.to_csv('test.csv')

data_file.close()

                                         Product_name Flipkart_price
0      R&P ENTERPRISE Analog 70 cm X 60 cm Wall Clock         [₹399]
1   ANNORA INTERNATIONAL Analog 25.5 cm X 25.5 cm ...         [₹335]
2               Kadio Analog 20 cm X 20 cm Wall Clock         [₹179]
3               Kadio Analog 20 cm X 20 cm Wall Clock         [₹179]
4              AJANTA Analog 32 cm X 32 cm Wall Clock         [₹474]
5          AJANTA Analog 26.5 cm X 26.5 cm Wall Clock         [₹268]
6          AJANTA Analog 20.5 cm X 20.5 cm Wall Clock         [₹279]
7   ANNORA INTERNATIONAL Analog 25 cm X 25 cm Wall...         [₹335]
8               Kadio Analog 33 cm X 24 cm Wall Clock         [₹249]
9           CAPIO ART Analog 26 cm X 26 cm Wall Clock         [₹292]
10  ANNORA INTERNATIONAL Analog 25.5 cm X 25.5 cm ...         [₹335]
11  pawan sut art and craft Analog 25 cm X 25 cm W...         [₹329]
12          Kadio Analog 24.5 cm X 24.5 cm Wall Clock         [₹229]
13     R&P ENTERPRISE Analog 65 cm

In [45]:
data = pd.read_csv("flipkart.csv", usecols = ['title','keySpecs','rating','ratingCount','price'])
data

,title,keySpecs,rating,ratingCount,price
0,Kadio Analog 20 cm X 20 cm Wall Clock,"['Size: 20 cm x 20 cm', 'Frame Material: Plast...",3.9,46605,179
1,ANNORA INTERNATIONAL Analog 25.5 cm X 25.5 cm ...,"['Size: 25.5 cm x 25.5 cm', 'Frame Material: W...",3.9,23,335
2,ANNORA INTERNATIONAL Analog 25.5 cm X 25.5 cm ...,"['Size: 25.5 cm x 25.5 cm', 'Mechanism: Quartz...",0.0,0,335
3,GrabBasket Analog 30 cm X 30 cm Wall Clock,"['Size: 30 cm x 30 cm', 'Mechanism: Quartz', '...",4.0,4358,385
4,Ingo Creation Analog 30 cm X 30 cm Wall Clock,"['Size: 30 cm x 30 cm', 'Frame Material: Woode...",3.9,1021,399
5,AJANTA Analog 31.3 cm X 31.3 cm Wall Clock,"['Size: 31.3 cm x 31.3 cm', 'Frame Material: G...",4.1,13490,408
6,BHATIA Analog Black Clock,"['Alarm', 'Height x Width 127 mm x 89 mm', 'An...",3.6,133,434
7,Basement Bazaar Analog 25 cm X 25 cm Wall Clock,"['Size: 25 cm x 25 cm', 'Mechanism: Quartz', '...",4.0,4740,369
8,Art Amori Analog 38 cm X 45 cm Wall Clock,"['Size: 45 cm x 38 cm', 'Mechanism: Quartz', '...",4.2,13938,598
9,Kadio Analog Silver Clock,"['Alarm', 'Height x Width 130 mm x 90 mm', 'An...",4.1,699,399


In [46]:
import requests
import time

In [47]:
flipkart=''
amazon=''
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

amazonlist = []
amazonName = []
i = 0
while i in range(len(names)):
    print(names[i])
    def amazon(name):
            try:
                global amazon
                name = " ".join(name.split(' ')[0:2])
                name1 = name.replace(" ","-")
                name2 = name.replace(" ","+")
                amazon=f'https://www.amazon.in/{name1}/s?k={name2}'
                res = requests.get(f'https://www.amazon.in/{name1}/s?k={name2}',headers=headers)
                print("\nSearching in amazon:")
                soup = BeautifulSoup(res.text,'html.parser')
                amazon_page = soup.select('.a-color-base.a-text-normal')
                amazon_page_length = int(len(amazon_page))
                for i in range(0,amazon_page_length):
                    name = name.upper()
                    amazon_name = soup.select('.a-color-base.a-text-normal')[i].getText().strip().upper()
                    if name in amazon_name[0:20]:
                        amazon_name = soup.select('.a-color-base.a-text-normal')[i].getText().strip().upper()
                        amazon_price = soup.select('.a-price-whole')[i].getText().strip().upper()
                        amazonlist.append(amazon_price)
                        print("Amazon:")
                        print(amazon_name)
                        amazonName.append(amazon_name)
                        print("₹"+amazon_price)
                        print("-----------------------")
                        break
                    else:
                        i+=1
                        i=int(i)
                        if i==amazon_page_length:
                            print("amazon : No product found!")
                            print("-----------------------")
                            amazon_price = '0'
                            amazonlist.append(amazon_price)
                            amazonName.append("No similar product")
                            break
                
                return amazon_price
            except:
                print("amazon: No product found!")
                print("-----------------------")
                amazon_price = '0'
                amazonlist.append(amazon_price)
                amazonName.append("No similar product")
            return amazon_price
    amazon_price = amazon(names[i])
    flipkart=''
    amazon=''
    i += 1
flip = flipPrices
idk = []
for i in range(len(flip)):
    try:
        x = flip[i].text.replace('₹','')
        print(x)
        idk.append(x)
    except:
        idk = test
df = pd.DataFrame(list(zip(names,idk,amazonName, amazonlist)), 
               columns =["Product_name_Flipkart","Flipkart_price",'Product_name_Amazon', 'Amazon_price'])
    
df.to_csv('flipkartandamazon.csv')
df

R&P ENTERPRISE Analog 70 cm X 60 cm Wall Clock

Searching in amazon:
amazon : No product found!
-----------------------
ANNORA INTERNATIONAL Analog 25.5 cm X 25.5 cm Wall Clock

Searching in amazon:
Amazon:
ANNORA INTERNATIONAL PACK OF 5 MAROON SAREE COVER/ CLOTH COVER / WARDROBE ORGANIZER (BLACK)
₹349
-----------------------
Kadio Analog 20 cm X 20 cm Wall Clock

Searching in amazon:
amazon : No product found!
-----------------------
Kadio Analog 20 cm X 20 cm Wall Clock

Searching in amazon:
amazon : No product found!
-----------------------
AJANTA Analog 32 cm X 32 cm Wall Clock

Searching in amazon:
Amazon:
AJANTA ANALOG GLASS WALL CLOCK (DARK GREEN, ORANGE, BLUE, 30 CM X 30 CM)
₹543
-----------------------
AJANTA Analog 26.5 cm X 26.5 cm Wall Clock

Searching in amazon:
Amazon:
AJANTA ANALOG GLASS WALL CLOCK (DARK GREEN, ORANGE, BLUE, 30 CM X 30 CM)
₹468
-----------------------
AJANTA Analog 20.5 cm X 20.5 cm Wall Clock

Searching in amazon:
Amazon:
AJANTA ANALOG GLASS WALL CLOCK 

,Product_name_Flipkart,Flipkart_price,Product_name_Amazon,Amazon_price
0,R&P ENTERPRISE Analog 70 cm X 60 cm Wall Clock,399,No similar product,0
1,ANNORA INTERNATIONAL Analog 25.5 cm X 25.5 cm ...,335,ANNORA INTERNATIONAL PACK OF 5 MAROON SAREE CO...,349
2,Kadio Analog 20 cm X 20 cm Wall Clock,179,No similar product,0
3,Kadio Analog 20 cm X 20 cm Wall Clock,179,No similar product,0
4,AJANTA Analog 32 cm X 32 cm Wall Clock,474,"AJANTA ANALOG GLASS WALL CLOCK (DARK GREEN, OR...",543
5,AJANTA Analog 26.5 cm X 26.5 cm Wall Clock,268,"AJANTA ANALOG GLASS WALL CLOCK (DARK GREEN, OR...",468
6,AJANTA Analog 20.5 cm X 20.5 cm Wall Clock,279,"AJANTA ANALOG GLASS WALL CLOCK (DARK GREEN, OR...",468
7,ANNORA INTERNATIONAL Analog 25 cm X 25 cm Wall...,335,ANNORA INTERNATIONAL PACK OF 5 MAROON SAREE CO...,359
8,Kadio Analog 33 cm X 24 cm Wall Clock,249,No similar product,0
9,CAPIO ART Analog 26 cm X 26 cm Wall Clock,292,CAPIO ART ENGINEERED WOOD HANGING POOJA TEMPLE...,298
